## DNA Search

In [1]:
from enum import IntEnum
from typing import Tuple, List

In [2]:
Nucleotide:IntEnum = IntEnum("Nucleotide", ("A", "C", "G", "T"))
Codon = Tuple[Nucleotide, Nucleotide, Nucleotide]
Gene = List[Codon]

In [3]:
gene_str: str = "ACGTGGCTCTCTAACGTACGTACGTACGGGGTTTATATATACCCTAGGACTCCCTTT"

In [4]:
def string_to_gene(s:str) -> Gene:
    gene:Gene = []
    for i in range(0, len(s), 3):
        if (i+2) >= len(s): #Checking for the end
            return gene
        codon:Codon = (Nucleotide[s[i]], Nucleotide[s[i+1]], Nucleotide[s[i+2]])
        gene.append(codon)
    return gene

In [5]:
my_gene: Gene = string_to_gene(gene_str)
print(my_gene)

[(<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>), (<Nucleotide.T: 4>, <Nucleotide.G: 3>, <Nucleotide.G: 3>), (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.C: 2>), (<Nucleotide.T: 4>, <Nucleotide.C: 2>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.A: 1>, <Nucleotide.C: 2>), (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.A: 1>), (<Nucleotide.C: 2>, <Nucleotide.G: 3>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>), (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.C: 2>), (<Nucleotide.G: 3>, <Nucleotide.G: 3>, <Nucleotide.G: 3>), (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.T: 4>), (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.T: 4>, <Nucleotide.A: 1>), (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.C: 2>), (<Nucleotide.C: 2>, <Nucleotide.C: 2>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.G: 3>, <Nucleotide.G: 3>), (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.T: 4

### Linear Search

In [6]:
def linear_contains(gene:Gene, key_codon:Codon) -> bool:
    for codon in gene:
        if codon == key_codon:
            return True
    return False

acg:Codon = (Nucleotide.A, Nucleotide.C, Nucleotide.G)
gat:Codon = (Nucleotide.G, Nucleotide.A, Nucleotide.T)
print(linear_contains(my_gene, acg)) # Should be True
print(linear_contains(my_gene, gat)) # Expect False

True
False


In [7]:
# Alternative using __contains__() method. we would never actually write the above.
print(acg in my_gene)
print(gat in my_gene)

True
False


### Binary Search

In [8]:
def binary_contains(gene:Gene, key_codon:Codon) -> bool:
    low:int = 0
    high:int = len(gene) - 1
    while low <= high: #i.e. while there still is a search space
        mid:int = (low+high)//2
        if gene[mid] < key_codon:
            low = mid+1
        elif gene[mid] > key_codon:
            high = mid-1
        else:
            return True
    return False

In [9]:
my_sorted_gene = sorted(my_gene)
print(binary_contains(my_sorted_gene, acg)) # Should be True
print(binary_contains(my_sorted_gene, gat)) # Should be False

True
False


### Generic Example

In [10]:
from __future__ import annotations
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional
from typing_extensions import Protocol
from heapq import heappush, heappop

In [14]:
T = TypeVar("T")

def linear_contains(iterable:Iterable[T], key:T) ->bool:
    for item in iterable:
        if item == key:
            return True
    return False

C = TypeVar("C", bound="Comparable")

class Comparable(Protocol):
    
    def __eq__(self, other:Any) -> bool:
        ...
    
    def __lt__(self:C, other:C) -> bool:
        ...
        
    def __gt__(self:C, other:C) -> bool:
        return (not self < other) and (self != other)
    
    def __le__(self:C, other:C) -> bool:
        return (self < other) or (self == other)
    
    def __ge__(self:C, other:C) -> bool:
        return (other < self) or (other == self)
    
def binary_contains(sequence:Sequence[C], key:C) -> bool:
    low:int = 0
    high:int = len(sequence) - 1
    while low <= high:
        mid:int = (low + high) //2
        if sequence[mid] < key:
            low = mid + 1
        elif key < sequence[mid]:
            high = mid - 1
        else:
            return True
    return False

In [15]:
print(linear_contains([1, 5, 15, 15, 15, 15, 20], 5)) # Expect True
print(binary_contains(["a", "d", "e", "f", "z"], "f")) # Expect True
print(binary_contains(["john", "mark", "ronald", "sarah"], "sheila")) # Expect False

True
True
False


## Maze Solving

In [16]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt
from generic_search import dfs, bfs, node_to_path, astar, Node

In [37]:
class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"
    
class MazeLocation(NamedTuple):
    row:int
    column:int
        
class Maze:
    def __init__(self, rows:int=10, columns:int=10, sparseness:float=0.2,
                 start:MazeLocation=MazeLocation(0,0),
                 goal:MazeLocation=MazeLocation(9,9)) -> None:
        self._rows:int = rows
        self._columns:int = columns
        self.start:MazeLocation = start
        self.goal:MazeLocation = goal
        self._grid:List[List[Cell]] = [[Cell.EMPTY for c in range(columns)] for r in range(rows)]
        self._randomly_fill(rows, columns, sparseness)
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL
        
    def _randomly_fill(self, rows:int, columns:int, sparseness:float) -> None:
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED
                    
    def __str__(self) -> str:
        output:str = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"
        return output
    
    def goal_test(self, ml:MazeLocation) -> bool:
        return ml == self.goal
    
    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations

    
    def mark(self, path: List[MazeLocation]) -> None:
        
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
            self._grid[self.start.row][self.start.column] = Cell.START
            self._grid[self.goal.row][self.goal.column] = Cell.GOAL
    
    def clear(self, path:List[MazeLocation]) -> None:
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
        

In [35]:
maze:Maze = Maze()
print(maze)

S  XX   X 
X  X      
   X  X   
X       X 
X X       
   X      
  X       
         X
 X       X
XXXX     G



### The DFS Algorithm

In [26]:
class Stack(Generic[T]):
    
    def __init__(self) -> None:
        self._container: List[T] = []
            
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item:T) -> None:
        self._container.append(item)
    
    def pop(self) -> T:
        return self._container.pop()
    
    def __repr__(self) -> str:
        return repr(self._container)
    
class Node(Generic[T]):
    
    def __init__(self, state:T, parent:Optional[Node], cost:float=0.0, heuristic:float=0.0) -> None:
        self.state:T = state
        self.parent:Optional[Node] = parent
        self.cost:float = cost
        self.heuristic:float = heuristic
    
    def __lt__(self, other:Node) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)
    
def dfs(initial:T, goal_test:Callable[[T], bool], successors:Callable[[T], List[T]]) -> Optional[Node[T]]:
    # Frontier is where we haven't gone yet.
    frontier:Stack[Node[T]] = Stack()
    frontier.push(Node(initial, None))
    # Explored is where we've been
    explored: Set[T] = {initial}
        
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we've found the goal, we're done
        if goal_test(current_state):
            return current_node
        # check where we can go next but haven't yet explored
        for child in successors(current_state):
            if child in explored: # we can skip this
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    
    return None # We tried but never found goal

def node_to_path(node:Node[T]) -> List[T]:
    path: List[T] = [node.state]
    # Work backwards from end to front
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    path.reverse()
    return path

Applying DFS to solve the maze

In [52]:
m:Maze = Maze()
print(m)
solution1:Optional[Node[MazeLocation]] = dfs(m.start, m.goal_test, m.successors)
if solution1 is None:
    print("No solution found using DFS")
else:
    path1: List[MazeLocation] = node_to_path(solution1)
    m.mark(path1)
    print(m)
    m.clear(path1)

S X      X
  X     X 
      XX  
  X X     
   XX   X 
 X X      
          
X         
          
 X  X   XG

S*X      X
 *X     X 
**    XX  
* X X     
***XX   X 
 X*X      
 **       
X*        
 *********
 X  X   XG



### Breadth-first Search

In [50]:
class Queue(Generic[T]):
    
    def __init__(self) -> None:
        self._container: Deque[T] = Deque()
            
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item:T) -> None:
        self._container.append(item)
        
    def pop(self) -> T:
        return self._container.popleft() #FIFO
    
    def __repr__(self) -> str:
        return repr(self._container)
    
def bfs(initial:T, goal_test:Callable[[T], bool], successors:Callable[[T], List[T]]) -> Optional[Node[T]]:
    # Frontier is where we haven't gone yet.
    frontier:Queue[Node[T]] = Queue()
    frontier.push(Node(initial, None))
    # Explored is where we've been
    explored: Set[T] = {initial}
        
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we've found the goal, we're done
        if goal_test(current_state):
            return current_node
        # check where we can go next but haven't yet explored
        for child in successors(current_state):
            if child in explored: # we can skip this
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    
    return None # We tried but never found goal




In [53]:
solution2:Optional[Node[MazeLocation]] = bfs(m.start, m.goal_test, m.successors)
if solution2 is None:
    print("No solution found using BFS")
else:
    path1: List[MazeLocation] = node_to_path(solution2)
    m.mark(path1)
    print(m)
    m.clear(path1)

S X      X
* X     X 
*     XX  
* X X     
*  XX   X 
*X X      
**        
X*        
 *********
 X  X   XG



### A* Search

In [55]:
class PriorityQueue(Generic[T]):
    
    def __init__(self) -> None:
        self._container:List [T] = []
            
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item:T) -> None:
        heappush(self._container, item) # In by priority
        
    def pop(self) -> T:
        return heappop(self._container) # out by priority
        
    def __repr__(self) -> str:
        return repr(self._container)
    

def euclidean_distance(goal:MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml:MazeLocation) -> float:
        xdist:int = ml.column - goal.column
        ydist:int = ml.row - goal.row
        return sqrt((xdist*xdist) + (ydist*ydist))
    
    return distance

def manhattan_distance(goal:MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml:MazeLocation) -> float:
        xdist:int = ml.column - goal.column
        ydist:int = ml.row - goal.row
        return abs(xdist) + abs(ydist)
    
    return distance



In [63]:
test_ml = MazeLocation(row=5, column=5)
print(m.goal)
ed = euclidean_distance(m.goal)
md = manhattan_distance(m.goal)
print(ed(test_ml), md(test_ml))

MazeLocation(row=9, column=9)
5.656854249492381 8


In [64]:
def astar(initial:T, goal_test:Callable[[T], bool], 
          successors:Callable[[T], List[T]],
          heuristic: Callable[[T], float]) -> Optional[Node[T]]:
    frontier: PriorityQueue[Node[T]] = PriorityQueue()
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    explored: Dict[T, float] = {initial:0.0}
    while not frontier.empty:
        current_node:Node[T] = frontier.pop()
        current_state: T = current_node.state
        # If we found goal, we're done
        if goal_test(current_state):
            return current_node
        # Check where we can go next and where we haven't explored
        for child in successors(current_state):
            new_cost:float = current_node.cost + 1 # Assumes a grid with equal step costs
            if child not in explored or explored[child] > new_cost:
                explored[child] = new_cost
                frontier.push(Node(child, current_node, new_cost, heuristic(child)))
    return None

In [66]:
distance:Callable[[MazeLocation], float] = manhattan_distance(m.goal)
solution3:Optional[Node[MazeLocation]] = astar(m.start, m.goal_test, m.successors, distance)
if solution3 is None:
    print("No solution found using A*")
else:
    path1: List[MazeLocation] = node_to_path(solution3)
    m.mark(path1)
    print(m)
    m.clear(path1)

S X      X
* X     X 
*     XX  
* X X     
***XX   X 
 X*X      
  ********
X        *
         *
 X  X   XG



## Missionaries and cannibals

In [93]:
MAX_NUM:int = 3
    
class MCState:
    
    def __init__(self, missionaries:int, cannibals:int, boat:bool) -> None:
        self.wm:int = missionaries
        self.wc:int = cannibals
        self.em:int = MAX_NUM - self.wm
        self.ec:int = MAX_NUM - self.wc
        self.boat:bool = boat
            
    def __str__(self) -> str:
        return "On the west bank there are {} missionaries and {} cannibals.\n"\
    "On the east bank there are {} missionaries and {} cannibals.\n"\
    "The boat is on the {} bank.".format(self.wm, self.wc, self.em, self.ec,
                                                    ("west" if self.boat else "east"))
    
    def goal_test(self) -> bool:
        return self.is_legal and self.em == MAX_NUM and self.ec==MAX_NUM
    
    @property
    def is_legal(self) -> bool:
        if self.wm < self.wc and self.wm > 0:
            return False
        if self.em < self.ec and self.em > 0:
            return False
        return True
    
    def successors(self) -> List[MCState]:
        sucs: List[MCState] = []
        if self.boat: # boat on west bank
            if self.wm > 1:
                sucs.append(MCState(self.wm - 2, self.wc, not self.boat))
            if self.wm > 0:
                sucs.append(MCState(self.wm - 1, self.wc, not self.boat))
            if self.wc > 1:
                sucs.append(MCState(self.wm, self.wc - 2, not self.boat))
            if self.wc > 0:
                sucs.append(MCState(self.wm, self.wc - 1, not self.boat))
            if (self.wc > 0) and (self.wm > 0):
                sucs.append(MCState(self.wm - 1, self.wc - 1, not self.boat))
        else: # boat on east bank
            if self.em > 1:
                sucs.append(MCState(self.wm + 2, self.wc, not self.boat))
            if self.em > 0:
                sucs.append(MCState(self.wm + 1, self.wc, not self.boat))
            if self.ec > 1:
                sucs.append(MCState(self.wm, self.wc + 2, not self.boat))
            if self.ec > 0:
                sucs.append(MCState(self.wm, self.wc + 1, not self.boat))
            if (self.ec > 0) and (self.em > 0):
                sucs.append(MCState(self.wm + 1, self.wc + 1, not self.boat))
        return [x for x in sucs if x.is_legal]

                
        return [x for x in sucs if x.is_legal]

def display_solution(path: List[MCState]):
    if len(path) == 0: # sanity check
        return
    old_state: MCState = path[0]
    print(old_state)
    for current_state in path[1:]:
        if current_state.boat:
            print("{} missionaries and {} cannibals moved from the east bank to the west bank.\n"
                  .format(old_state.em - current_state.em, old_state.ec - current_state.ec))
        else:
            print("{} missionaries and {} cannibals moved from the west bank to the east bank.\n"
                  .format(old_state.wm - current_state.wm, old_state.wc - current_state.wc))
        print(current_state)
        old_state = current_state



In [94]:
mc = MCState(3, 3, True)
print(mc, [suc.__str__() for suc in mc.successors()])

On the west bank there are 3 missionaries and 3 cannibals.
On the east bank there are 0 missionaries and 0 cannibals.
The boat is on the west bank. ['On the west bank there are 3 missionaries and 1 cannibals.\nOn the east bank there are 0 missionaries and 2 cannibals.\nThe boat is on the east bank.', 'On the west bank there are 3 missionaries and 2 cannibals.\nOn the east bank there are 0 missionaries and 1 cannibals.\nThe boat is on the east bank.', 'On the west bank there are 2 missionaries and 2 cannibals.\nOn the east bank there are 1 missionaries and 1 cannibals.\nThe boat is on the east bank.']


In [95]:
start:MCState = MCState(MAX_NUM, MAX_NUM, True)
solution:Optional[Node[MCState]] = bfs(start, MCState.goal_test, MCState.successors)
if solution is None:
    print("No solution found")
else:
    path:List[MCState] = node_to_path(solution)
    display_solution(path)

On the west bank there are 3 missionaries and 3 cannibals.
On the east bank there are 0 missionaries and 0 cannibals.
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west bank to the east bank.

On the west bank there are 3 missionaries and 1 cannibals.
On the east bank there are 0 missionaries and 2 cannibals.
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east bank to the west bank.

On the west bank there are 3 missionaries and 2 cannibals.
On the east bank there are 0 missionaries and 1 cannibals.
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west bank to the east bank.

On the west bank there are 3 missionaries and 0 cannibals.
On the east bank there are 0 missionaries and 3 cannibals.
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east bank to the west bank.

On the west bank there are 3 missionaries and 1 cannibals.
On the east bank there are 0 missionaries and 2 c

In [96]:
mc = MCState(3, 2, True)
print(mc, [suc.__str__() for suc in mc.successors()])

On the west bank there are 3 missionaries and 2 cannibals.
On the east bank there are 0 missionaries and 1 cannibals.
The boat is on the west bank. ['On the west bank there are 2 missionaries and 2 cannibals.\nOn the east bank there are 1 missionaries and 1 cannibals.\nThe boat is on the east bank.', 'On the west bank there are 3 missionaries and 0 cannibals.\nOn the east bank there are 0 missionaries and 3 cannibals.\nThe boat is on the east bank.', 'On the west bank there are 3 missionaries and 1 cannibals.\nOn the east bank there are 0 missionaries and 2 cannibals.\nThe boat is on the east bank.']


In [99]:
start:MCState = MCState(3, 0, True)
solution:Optional[Node[MCState]] = bfs(start, MCState.goal_test, MCState.successors)
if solution is None:
    print("No solution found")
else:
    path:List[MCState] = node_to_path(solution)
    display_solution(path)

No solution found
